<a href="https://colab.research.google.com/github/hamdi-trikii/Data-Management-in-Python/blob/main/consolidating_Employee_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

You just got hired as the first and only data practitioner at a small business experiencing exponential growth. The company needs more structured processes, guidelines, and standards. Your first mission is to structure the human resources data. The data is currently scattered across teams and files and comes in various formats: Excel files, CSVs, JSON files...

You'll work with the following data in the `datasets` folder:
- __Office addresses__
    - Saved in `office_addresses.csv`.
    - If the value for office is `NaN`, then the employee is remote.
- __Employee addresses__
    - Saved on the first tab of `employee_information.xlsx`.
- __Employee emergency contacts__
    - Saved on the second tab of `employee_information.xlsx`; this tab is called `emergency_contacts`.
    - However, this sheet was edited at some point, and ***the headers were removed***! The HR manager let you know that they should be: `employee_id`, `last_name`, `first_name`, `emergency_contact`, `emergency_contact_number`, and `relationship`.
- __Employee roles, teams, and salaries__
    - This information has been exported from the company's human resources management system into a JSON file titled `employee_roles.json`. Here are the first few lines of that file:
```
{"A2R5H9":
  {
    "title": "CEO",
    "monthly_salary": "$4500",
    "team": "Leadership"
  },
 ...
}
```

In [2]:
import pandas as pd
# Start coding here...

offices= pd.read_csv('datasets/office_addresses.csv')

addresses = pd.read_excel('datasets/employee_information.xlsx')

emergency_contacts_header = ["employee_id", "last_name", "first_name", "emergency_contact", "emergency_contact_number", "relationship"]

emergency_contacts = pd.read_excel('datasets/employee_information.xlsx', sheet_name="emergency_contacts", header=None , names=emergency_contacts_header)

roles = pd.read_json('datasets/employee_roles.json', orient='index')
print(offices)
print(addresses)
print(emergency_contacts)
print(roles)

          office office_country    office_city   office_street  \
0  Leuven Office             BE         Leuven  Martelarenlaan   
1     ESB Office             US  New York City    Fifth Avenue   
2  WeWork Office             GB         London      Old Street   

   office_street_number  
0                    38  
1                   350  
2                   207  
  employee_id employee_last_name employee_first_name employee_country  \
0      A2R5H9             Hunman                 Jax               BE   
1      H8K0L6               Siff                Tara               GB   
2      G4R7V0              Sagal               Gemma               US   
3      M1Z7U9             Coates                 Tig               FR   

  employee_city      employee_street  employee_street_number  
0        Leuven          Grote Markt                       9  
1        London         Baker Street                     221  
2      New-York         Perry Street                      66  
3         Par

Merging the DataFrames

In [5]:
#merging addresses with offices
employees = addresses.merge(offices, left_on="employee_country", right_on="office_country", how="left")
print(employees)

# Merging employees with roles
employees = employees.merge(roles, left_on="employee_id", right_index=True)
print(employees)

#Merging employees with emergency_contacts
employees = employees.merge(emergency_contacts, on="employee_id", how="left")
print(employees)


  employee_id employee_last_name employee_first_name employee_country  \
0      A2R5H9             Hunman                 Jax               BE   
1      H8K0L6               Siff                Tara               GB   
2      G4R7V0              Sagal               Gemma               US   
3      M1Z7U9             Coates                 Tig               FR   

  employee_city      employee_street  employee_street_number         office  \
0        Leuven          Grote Markt                       9  Leuven Office   
1        London         Baker Street                     221  WeWork Office   
2      New-York         Perry Street                      66     ESB Office   
3         Paris  Rue de l'Université                       7            NaN   

  office_country    office_city   office_street  office_street_number  
0             BE         Leuven  Martelarenlaan                  38.0  
1             GB         London      Old Street                 207.0  
2             US  New 


Cleaning and formatting the data


In [6]:
#Filling null values
for column in employees.columns:
    if column.startswith("office"):
        employees[column].fillna("Remote", inplace=True)
print(employees)

#Subsetting and reindexing the DataFrame
final_columns = ['employee_id', 'employee_first_name', 'employee_last_name', 'employee_country', 'employee_city', 'employee_street', 'employee_street_number', 'emergency_contact', 'emergency_contact_number', 'relationship', 'monthly_salary', 'team', 'title', 'office', 'office_country', 'office_city', 'office_street', 'office_street_number']

employees_final = employees[final_columns]
employees_final.set_index("employee_id", inplace=True)
print(employees_final)

  employee_id employee_last_name employee_first_name employee_country  \
0      A2R5H9             Hunman                 Jax               BE   
1      H8K0L6               Siff                Tara               GB   
2      G4R7V0              Sagal               Gemma               US   
3      M1Z7U9             Coates                 Tig               FR   

  employee_city      employee_street  employee_street_number         office  \
0        Leuven          Grote Markt                       9  Leuven Office   
1        London         Baker Street                     221  WeWork Office   
2      New-York         Perry Street                      66     ESB Office   
3         Paris  Rue de l'Université                       7         Remote   

  office_country    office_city   office_street office_street_number  \
0             BE         Leuven  Martelarenlaan                 38.0   
1             GB         London      Old Street                207.0   
2             US  New 